In [22]:
import json
import pandas as pd

# Cargar el archivo JSON
with open('meteorología_2023.json') as f:
    data = json.load(f)

# Inicializar una lista para almacenar las filas de la tabla
rows = []

# Unidades para las medidas (opcional para clarificar)
units = data['pollutionMeasurements']['unit']

# Iterar sobre las fechas y horas en el JSON
for datetime, measurements in data['pollutionMeasurements']['date'].items():
    # Para cada tipo de medición (RH, TMP, WDR, WSP, etc.)
    for measure_type, stations in measurements.items():
        # Para cada estación dentro de cada tipo de medición
        for station, value in stations.items():
            # Buscar si ya hay una fila para esta combinación de datetime y estación
            row = next((r for r in rows if r['fecha_hora'] == datetime and r['estacion'] == station), None)
            if row is None:
                row = {'fecha_hora': datetime, 'estacion': station}
                rows.append(row)
            # Añadir la medición al diccionario
            row[measure_type] = value

# Convertir a DataFrame
df = pd.DataFrame(rows)

# Guardar en un archivo CSV
df.to_csv('mediciones.csv', index=False)

# Mostrar el DataFrame resultante
print(df)



             fecha_hora estacion   RH   TMP  WDR  WSP  PBa
0      2023-01-01 01:00      ACO   81   9.3  319  1.3  NaN
1      2023-01-01 01:00      AJU  100   2.5   36  2.4  NaN
2      2023-01-01 01:00      MON   56  13.9  177  1.8  NaN
3      2023-01-01 01:00      CHO                       NaN
4      2023-01-01 01:00      CUA   61  10.9  258  1.7  NaN
...                 ...      ...  ...   ...  ...  ...  ...
80635  2023-04-30 24:00      INN   44   6.8  153  1.3  NaN
80636  2023-04-30 24:00      GAM   37    20  273  1.6  NaN
80637  2023-04-30 24:00      LAA   22  20.4  282  1.1  NaN
80638  2023-04-30 24:00      FAR  NaN   NaN  249  2.2  NaN
80639  2023-04-30 24:00      SAC  NaN   NaN  272  1.4  NaN

[80640 rows x 7 columns]


In [5]:
import json
import pandas as pd

# Cargar el archivo JSON
with open('meteorología_2023.json') as f:
    data = json.load(f)

# Inicializar una lista para almacenar las filas de la tabla
rows = []

# Unidades para las medidas (opcional para clarificar)
units = data['pollutionMeasurements']['unit']

# Iterar sobre las fechas y horas en el JSON
for datetime, measurements in data['pollutionMeasurements']['date'].items():
    # Para cada tipo de medición (RH, TMP, WDR, WSP, etc.)
    for measure_type, stations in measurements.items():
        # Para cada estación dentro de cada tipo de medición
        for station, value in stations.items():
            # Buscar si ya hay una fila para esta combinación de datetime y estación
            row = next((r for r in rows if r['fecha_hora'] == datetime and r['estacion'] == station), None)
            if row is None:
                row = {'fecha_hora': datetime, 'estacion': station}
                rows.append(row)
            # Añadir la medición al diccionario
            row[measure_type] = value

# Convertir a DataFrame
df = pd.DataFrame(rows)

# Guardar en un archivo CSV
df.to_csv('mediciones.csv', index=False)

# Mostrar el DataFrame resultante
print(df)


           Fecha y Hora Estación   RH   TMP  WDR  WSP  PBa
0      2023-01-01 01:00      ACO   81   9.3  319  1.3  NaN
1      2023-01-01 01:00      AJU  100   2.5   36  2.4  NaN
2      2023-01-01 01:00      MON   56  13.9  177  1.8  NaN
3      2023-01-01 01:00      CHO                       NaN
4      2023-01-01 01:00      CUA   61  10.9  258  1.7  NaN
...                 ...      ...  ...   ...  ...  ...  ...
80635  2023-04-30 24:00      INN   44   6.8  153  1.3  NaN
80636  2023-04-30 24:00      GAM   37    20  273  1.6  NaN
80637  2023-04-30 24:00      LAA   22  20.4  282  1.1  NaN
80638  2023-04-30 24:00      FAR  NaN   NaN  249  2.2  NaN
80639  2023-04-30 24:00      SAC  NaN   NaN  272  1.4  NaN

[80640 rows x 7 columns]


In [8]:
pip install pandas pymysql


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 320.0 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 551.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
import json
import pandas as pd
import pymysql

# Cargar el archivo JSON
with open('meteorología_2023.json') as f:
    data = json.load(f)

# Inicializar una lista para almacenar las filas de la tabla
rows = []

# Iterar sobre las fechas y horas en el JSON
for datetime, measurements in data['pollutionMeasurements']['date'].items():
    # Para cada tipo de medición (RH, TMP, WDR, WSP, etc.)
    for measure_type, stations in measurements.items():
        # Para cada estación dentro de cada tipo de medición
        for station, value in stations.items():
            # Buscar si ya hay una fila para esta combinación de datetime y estación
            row = next((r for r in rows if r['fecha_hora'] == datetime and r['estacion'] == station), None)
            if row is None:
                row = {'fecha_hora': datetime, 'estacion': station}
                rows.append(row)
            # Añadir la medición al diccionario
            row[measure_type] = value

# Convertir a DataFrame
df = pd.DataFrame(rows)

# Reemplazar NaN por None
df = df.applymap(lambda x: None if pd.isna(x) else x)

# Comprobación adicional de los datos convertidos
print("Data tuples:", df.head())

# Conectar a MySQL
try:
    connection = pymysql.connect(
        host='34.72.69.10',
        user='bd_conagua',
        password='HkW0eiFxfT5iUv31',
        database='conagua'
    )
    cursor = connection.cursor()

    # Verificar la estructura de la tabla
    cursor.execute("DESCRIBE mediciones")
    columns = [column[0] for column in cursor.fetchall()]
    print("Columns in the mediciones table:", columns)

    # Asegurarse de que las columnas coinciden
    if not all(col in columns for col in df.columns):
        raise ValueError("Las columnas especificadas no coinciden con la estructura de la tabla 'mediciones'.")

    # Preparar los datos para la inserción
    data_tuples = [tuple(x) for x in df.to_numpy()]
    cols = ",".join([str(i) for i in df.columns.tolist()])

    # Insertar los datos
    insert_query = f"INSERT INTO mediciones ({cols}) VALUES ({', '.join(['%s'] * len(df.columns))})"
    cursor.executemany(insert_query, data_tuples)
    connection.commit()

except pymysql.MySQLError as e:
    print(f"Error al conectar a la base de datos: {e}")
except ValueError as ve:
    print(f"Error: {ve}")
finally:
    if connection:
        cursor.close()
        connection.close()


C:\Users\danws\AppData\Local\Temp\ipykernel_20676\2952902499.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if pd.isna(x) else x)


Data tuples:          fecha_hora estacion   RH   TMP  WDR  WSP   PBa
0  2023-01-01 01:00      ACO   81   9.3  319  1.3  None
1  2023-01-01 01:00      AJU  100   2.5   36  2.4  None
2  2023-01-01 01:00      MON   56  13.9  177  1.8  None
3  2023-01-01 01:00      CHO                       None
4  2023-01-01 01:00      CUA   61  10.9  258  1.7  None
Columns in the mediciones table: ['fecha_hora', 'estacion', 'RH', 'TMP', 'WDR', 'WSP']
Error: Las columnas especificadas no coinciden con la estructura de la tabla 'mediciones'.


In [28]:
import pymysql
# Conectar a MySQL
connection = pymysql.connect(
    host='34.72.69.10',
    user='bd_conagua',
    password='HkW0eiFxfT5iUv31',
    database='conagua'
)

cursor = connection.cursor()

# Crear la tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS mediciones (
    fecha_hora DATETIME,
    estacion VARCHAR(50),
    RH FLOAT,
    TMP FLOAT,
    WDR FLOAT,
    WSP FLOAT,
    PRIMARY KEY (fecha_hora, estacion)
)
"""
cursor.execute(create_table_query)
connection.commit()


In [29]:
# Preparar los datos para la inserción
data_tuples = [tuple(x) for x in df.to_numpy()]
cols = ",".join([str(i) for i in df.columns.tolist()])

# Insertar los datos
insert_query = f"INSERT INTO mediciones ({cols}) VALUES (%s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_query, data_tuples)
connection.commit()

# Cerrar la conexión
cursor.close()
connection.close()


TypeError: not all arguments converted during string formatting

In [30]:
import pymysql
import pandas as pd
import numpy as np

# Supongamos que 'data' es tu DataFrame de Pandas
data = pd.DataFrame({
    'col1': [1, 2, np.nan],
    'col2': ['A', 'B', 'C'],
    'col3': [10.5, np.nan, 12.3]
})

# Reemplazar NaN por None
data = data.map(lambda x: None if pd.isna(x) else x)

# Convertir los datos a una lista de tuplas
data_tuples = [tuple(None if pd.isna(x) else x for x in row) for row in data.itertuples(index=False, name=None)]

# Comprobación adicional de los datos convertidos
print("Data tuples:", data_tuples)

# Conectar a MySQL
try:
    connection = pymysql.connect(
        host='34.72.69.10',
        user='bd_conagua',
        password='HkW0eiFxfT5iUv31',
        database='conagua'
    )
    cursor = connection.cursor()

    # Verificar la estructura de la tabla
    cursor.execute("DESCRIBE mediciones")
    columns = [column[0] for column in cursor.fetchall()]
    print("Columns in the mediciones table:", columns)

    # Asumiendo que tu tabla tiene columnas correspondientes
    cols = 'col1, col2, col3'
    if not all(col in columns for col in cols.split(', ')):
        raise ValueError("Las columnas especificadas no coinciden con la estructura de la tabla 'mediciones'.")

    insert_query = f"INSERT INTO mediciones ({cols}) VALUES (%s, %s, %s)"

    # Insertar los datos
    cursor.executemany(insert_query, data_tuples)
    connection.commit()

except pymysql.MySQLError as e:
    print(f"Error al conectar a la base de datos: {e}")
except ValueError as ve:
    print(f"Error: {ve}")
finally:
    if connection:
        connection.close()



Data tuples: [(1.0, 'A', 10.5), (2.0, 'B', None), (None, 'C', 12.3)]
Columns in the mediciones table: ['fecha_hora', 'estacion', 'RH', 'TMP', 'WDR', 'WSP']
Error: Las columnas especificadas no coinciden con la estructura de la tabla 'mediciones'.
